# PyVista isosurface demos

In [1]:
import pyvista as pv
pv.set_jupyter_backend('trame')
from viz import build_validation_scenes, isosurface_with_clip,  generate_reg_mesh, isolines_with_clip
from geometry import load_obj_mesh
from viz import build_validation_scene_specs, visualize_pointed_vertices

In [2]:
levels = [ 10, 50, 100.0, 500.0, 1000.0]
scenes =  build_validation_scenes(reverse_faces=True)

In [3]:
#  single triangle
plotter = isosurface_with_clip(scenes[0], 
    levels=levels, res=100)
plotter.show()

Widget(value='<iframe src="http://localhost:55845/index.html?ui=P_0x119df17f0_0&reconnect=auto" class="pyvista…

In [7]:
#  single triangle
plotter = isolines_with_clip(scenes[0], 
    levels=levels, res=100)
#plotter.show()

Widget(value='<iframe src="http://localhost:55845/index.html?ui=P_0x123f274d0_4&reconnect=auto" class="pyvista…

In [8]:
# two orthogonal triangles
plotter = isosurface_with_clip(scenes[1], levels=levels, res=100)
plotter.show()

Widget(value='<iframe src="http://localhost:55845/index.html?ui=P_0x123f27d90_5&reconnect=auto" class="pyvista…

In [11]:
# 4-triangle saddle
plotter = isosurface_with_clip(scenes[2], levels=levels, res=100)
plotter.show()

Widget(value='<iframe src="http://localhost:50567/index.html?ui=P_0x12d262fd0_4&reconnect=auto" class="pyvista…

In [12]:
# 2 x 2 grid of 8 coplanar triangles, should look the same as 2 triangles, basic refinement test
mesh = generate_reg_mesh(lambda u, v: [u, v, 0], n=2)
plotter = isosurface_with_clip(mesh, levels=[10, 100, 200, 500, 1000], res=60)
plotter.show()

Widget(value='<iframe src="http://localhost:50567/index.html?ui=P_0x12d263c50_5&reconnect=auto" class="pyvista…

In [13]:
# triangluated paraboloid segment, approximated by 8 x 8 grid, 128 triangles
mesh = generate_reg_mesh(lambda u, v: [u, v, (u-0.501)**2 + (v-0.501)**2], n=8)
plotter = isosurface_with_clip(mesh, levels=[10, 50, 100, 200, 500, 1000], res=60)
plotter.show()

Widget(value='<iframe src="http://localhost:50567/index.html?ui=P_0x13c8b8a50_6&reconnect=auto" class="pyvista…

In [14]:
# sharp corner, 4 triangles, 2 on each side
mesh = generate_reg_mesh(lambda u, v: [u, v, abs(10.01*u-5.01)], n=2)
plotter = isosurface_with_clip(mesh, levels=[10, 50, 100, 200, 500, 1000], res=150)
plotter.show()

Widget(value='<iframe src="http://localhost:50567/index.html?ui=P_0x13c8b9590_7&reconnect=auto" class="pyvista…

In [ ]:
# ~ 280 triangle bunny, level sets chosen to look at the far behavior
mesh = load_obj_mesh("tests/Bunny-LowPoly.obj")
plotter = isosurface_with_clip(mesh, res=40, levels=[0.001, 0.01, 0.1])
plotter.show()

In [10]:
scenes = build_validation_scene_specs(reverse_faces=False)
scenes_flipped = build_validation_scene_specs(reverse_faces=True)

In [47]:
visualize_pointed_vertices(scenes[4].mesh)  # tetrahedron

Widget(value='<iframe src="http://localhost:50567/index.html?ui=P_0x18bc24cd0_26&reconnect=auto" class="pyvist…

In [25]:
visualize_pointed_vertices(scenes[5].mesh)  # cube_face_centers

Widget(value='<iframe src="http://localhost:50567/index.html?ui=P_0x13c8bb750_13&reconnect=auto" class="pyvist…

In [27]:
visualize_pointed_vertices(scenes[6].mesh)  # double_cone_nonconvex_out

Widget(value='<iframe src="http://localhost:50567/index.html?ui=P_0x154e3afd0_15&reconnect=auto" class="pyvist…

In [5]:
visualize_pointed_vertices(scenes_flipped[4].mesh)

Widget(value='<iframe src="http://localhost:57962/index.html?ui=P_0x1169aa490_1&reconnect=auto" class="pyvista…

In [37]:
# flipped double_cone_nonconvex_* scenes

visualize_pointed_vertices(scenes_flipped[5].mesh)

Widget(value='<iframe src="http://localhost:50567/index.html?ui=P_0x18bc25810_19&reconnect=auto" class="pyvist…

In [38]:
visualize_pointed_vertices(scenes_flipped[6].mesh)

Widget(value='<iframe src="http://localhost:50567/index.html?ui=P_0x18bc26490_20&reconnect=auto" class="pyvist…

In [53]:
# tetrahedron with normals pointing inwards (default of what scene_specs generates for no good reason)
plotter = isosurface_with_clip(scenes[4].mesh, 
    levels=[100, 200, 500, 1000], res=100, alpha=0.01, 
    one_sided=True,  
    include_faces=True,
    include_edges=True, 
    include_vertices= True,
    show_mesh=True,                              
 )
plotter.show()

Widget(value='<iframe src="http://localhost:50567/index.html?ui=P_0x1dc2c87d0_31&reconnect=auto" class="pyvist…

In [49]:
# cube with face centers inserted, and one pushed inward, also with normals pointing inside
plotter = isosurface_with_clip(scenes[5].mesh, 
    levels=[10, 20,  50, 100, 200, 500, 1000], res=60, alpha=0.1, 
    one_sided=True,  
    include_faces=True,
    include_edges=True, 
    include_vertices= True,
    show_mesh=True
 )
plotter.show()

Widget(value='<iframe src="http://localhost:50567/index.html?ui=P_0x1b278e850_28&reconnect=auto" class="pyvist…

In [58]:
# tetrahedron with normals pointing outside
plotter = isosurface_with_clip(scenes_flipped[4].mesh, 
    levels=[100, 200, 500, 1000], res=100, alpha=0.01, 
    one_sided=True,  
    include_faces=True,
    include_edges=True, 
    include_vertices= True,
    show_mesh=True,                              
 )
plotter.show()

Widget(value='<iframe src="http://localhost:50567/index.html?ui=P_0x20b118550_38&reconnect=auto" class="pyvist…

In [59]:
# cube with face centers inserted, and one pushed inward, also with normals pointing outward
plotter = isosurface_with_clip(scenes_flipped[5].mesh, 
    levels=[10, 20,  50, 100, 200, 500, 1000], res=60, alpha=0.1, 
    one_sided=True,  
    include_faces=True,
    include_edges=True, 
    include_vertices= True,
    show_mesh=True
 )
plotter.show()

Widget(value='<iframe src="http://localhost:50567/index.html?ui=P_0x20b118e10_39&reconnect=auto" class="pyvist…

In [21]:
# double nonconvex cone with outside-pointing normals: weird interior aliasing I do not fully  understand, need to check with 2d plotting
# I am guessing it is due to mesh/alignment and atempts to resolve the isolines on the "inside" part of the surface, where the "leaked" potential has discontinuities
# may be a bug but no clear reason, all individual components look reasonable but are very aliased
plotter =  isosurface_with_clip(scenes[6].mesh, 
    levels=[200], res=100, alpha=0.01, 
    one_sided=True,  
    include_faces=False,
    include_edges=True, 
    include_vertices= False,
    show_mesh=True
 )
plotter.show()

/Users/dzorin/src/hiorder_gcp_3d/potential.py:60: RuntimeWarning: divide by zero encountered in divide
  return H(np.asarray(t, dtype=float) / alpha)


Widget(value='<iframe src="http://localhost:57962/index.html?ui=P_0x169a70e10_15&reconnect=auto" class="pyvist…

In [18]:
plotter =  isolines_with_clip(scenes[6].mesh, 
    levels=[10, 100, 200, 500,1000], res=300, alpha=0.01, 
    one_sided=True,  
    include_faces=False,
    include_edges=False, 
    include_vertices= True,
    show_mesh=True
 )


Widget(value='<iframe src="http://localhost:55845/index.html?ui=P_0x163bcce10_13&reconnect=auto" class="pyvist…

In [14]:
# double nonconvex cone with outward-pointing normals
plotter = isosurface_with_clip(scenes_flipped[6].mesh, 
    levels=[   50, 100, 200, 500], res=100, alpha=0.01, 
    one_sided=True,  
    include_faces=True,
    include_edges=True, 
    include_vertices= True,
    show_mesh=True
 )
plotter.show()

Widget(value='<iframe src="http://localhost:57962/index.html?ui=P_0x13c17ce10_8&reconnect=auto" class="pyvista…